# Update Items

### Import Libraries

In [1]:
import glob
import os
import json
import xml.etree.ElementTree as ET
import requests
from urllib import parse
import argparse
import urllib.request
import configparser
import tqdm
import time

### Get configuration file

In [2]:
config = configparser.ConfigParser()
config.sections()
config.read('../config/api.ini')
client_id = config.get('main', 'client_id')
client_secret = config.get('main', 'client_secret')
endpoint = config.get('main', 'endpoint')

### Connect to the Api

In [3]:
params = {
    'key_identity': client_id,
    'key_credential': client_secret
}

access_url = endpoint
print("Connected to:    ", access_url)

Connected to:     http://localhost:8080/api/


### Navigate the xml and converts it to json

In [4]:
dictionary = {}
fileName = ''

for file in glob.glob("../input/*"):
    root = ET.parse(file).getroot()
    fileName = str(os.path.basename(file))
    
    for group in root.findall('family/group'):
        groupName = group.attrib['name']
        item = group.findall('item')
        
        for value in item:
            fieldName = value.attrib['name']         
            matchingName = groupName +"_"+ fieldName
            fieldValue = value.text
            
            #fix getty links 
            if "getty" in str(fieldValue) and 'subjectid=' in str(fieldValue) and '[' in str(fieldValue) and '[' in str(fieldValue):
                name  = fieldValue.split('[')[0].split(']')[0]
                url  = fieldValue.split('[')[1].split(']')[0]

                fieldValue = name + "; http://vocab.getty.edu/page/tgn/" +  replaceElement

                if  'subjectid=' in str(url):
                    url  = url.split('subjectid=')[1]
                    fieldValue = name + "; http://vocab.getty.edu/page/tgn/" +  url
                    
            cleanedText = str(fieldValue).replace("None", "").replace("\n", "").replace("\t", " ").replace('"', '').replace("'", "")
            
            # Creating/appending to a JSON structure
            if fileName not in dictionary:
                dictionary[fileName] = { matchingName :  [cleanedText] }
            else:
                if matchingName not in dictionary[fileName]:
                    dictionary[fileName][matchingName] = [cleanedText]
                else:
                    dictionary[fileName][matchingName].append(cleanedText)

#### Structure a JSON-LD item

In [109]:
def structureField(dict_type, property_id, value):
    return { dict_type: [ { "type": "literal", "property_id": property_id, "@value": value } ]}

def structureLabel(dict_type, property_id, value, **lang):
    
    if lang:
        return {"type": dict_type, "property_id": property_id, "@value": value, "@language": lang}
    else:
        return {"type": dict_type, "property_id": property_id, "@value": value}

### Import multiple items

In [110]:
url = endpoint + 'items?page=all'

print(url)
response = json.loads(urllib.request.urlopen(url).read().decode('utf-8'))

pbar = tqdm(range(len(response)), desc='description')

for i in pbar:
    try:
        identifier = response[i]['dcterms:identifier'][0]['@value']
        resourceId = response[i]['o:id']

        if identifier in dictionary.keys():
            
            # --------- dcterms:identifier --------- #
            dcidentifier = structureField('dcterms:identifier', 10, identifier)
   
            # --------- dcterms:extent --------- #
            
            dictionaryID = 'dcterms:extent'
            propertyID = 25
            dcextent =  { dictionaryID: [] }
            
            paintingSize = dictionary[identifier]['Objekt_Gemälde Maße max. (H x B x T in cm)'][0].split('x')
            frameSize = dictionary[identifier]['Objekt_Gemälde Maße max. (H x B x T in cm)'][0].split('x')

            try:
                paintingHeight = 'Canvas height ' + paintingSize[0] + ' cm'
                elem = structureLabel('literal', propertyID, paintingHeight)
                dcextent[dictionaryID].append(elem)
            except:
                pass

            try:
                paintingWidth =  'Canvas width ' + paintingSize[1] + ' cm'
                elem = structureLabel('literal', propertyID, paintingWidth)
                dcextent[dictionaryID].append(elem)
            except:
                pass

            try:
                paintingDepth =  'Canvas depth ' + paintingSize[2] + ' cm'
                elem = structureLabel('literal', propertyID, paintingDepth)
                dcextent[dictionaryID].append(elem)

            except:
                pass

            try:
                frameHeight = 'Frame height ' + frameSize[0] + ' cm'
                elem = structureLabel('literal', propertyID, frameHeight)
                dcextent[dictionaryID].append(elem)
            except:
                pass

            try:
                frameWidth =  'Frame width ' + frameSize[1] + ' cm'
                elem = structureLabel('literal', propertyID, frameWidth)
                dcextent[dictionaryID].append(elem)
            except:
                pass

            try:
                frameDepth =  'Frame depth ' + frameSize[2] + ' cm'
                elem = structureLabel('literal', propertyID, frameDepth)
                dcextent[dictionaryID].append(elem)

            except:
                pass
          
            # --------- dcterms:format --------- #
            
            dictionaryID = 'dcterms:format'
            propertyID = 9
            
            dcformat =  { dictionaryID: [] }
            
            paintingMaterial = dictionary[identifier]['Objekt_Gemälde Material'][0].split(';')
            
            for material in paintingMaterial:
                if(material != ""):
                    elem = structureLabel('literal', propertyID, material, 'de')
                    dcformat[dictionaryID].append(elem)
                    
            # --------- dcterms:format --------- #
            
        
            # Merge all jsons previously defined
            patch = {**dcidentifier, **dcextent, **dcformat }
            
            
            # Merge with the response json
            response[i].update(patch)
            postRequest = json.dumps(response[i])
            
        
            putUrl = endpoint + 'items/' +  str(resourceId)
            headers = {"Content-Type": "application/json"}
            r = requests.patch(putUrl, data=postRequest, params = params, headers=headers)
            
            time.sleep(.25)
            pbar.set_description("ITEM: " + str(identifier) +  "| ID: "+  str(resourceId))
    except Exception as e:
        print(e)

http://localhost:8080/api/items?page=all


description:   0%|          | 0/641 [00:00<?, ?it/s]

structureLabel() takes 3 positional arguments but 4 were given
structureLabel() takes 3 positional arguments but 4 were given
structureLabel() takes 3 positional arguments but 4 were given
structureLabel() takes 3 positional arguments but 4 were given
structureLabel() takes 3 positional arguments but 4 were given
structureLabel() takes 3 positional arguments but 4 were given
structureLabel() takes 3 positional arguments but 4 were given
structureLabel() takes 3 positional arguments but 4 were given
structureLabel() takes 3 positional arguments but 4 were given
structureLabel() takes 3 positional arguments but 4 were given
structureLabel() takes 3 positional arguments but 4 were given
structureLabel() takes 3 positional arguments but 4 were given
structureLabel() takes 3 positional arguments but 4 were given
structureLabel() takes 3 positional arguments but 4 were given


ITEM: R_15_594| ID: 48:   2%|▏         | 15/641 [00:00<00:36, 17.32it/s]

structureLabel() takes 3 positional arguments but 4 were given
structureLabel() takes 3 positional arguments but 4 were given
structureLabel() takes 3 positional arguments but 4 were given
structureLabel() takes 3 positional arguments but 4 were given
structureLabel() takes 3 positional arguments but 4 were given


...

structureLabel() takes 3 positional arguments but 4 were given
structureLabel() takes 3 positional arguments but 4 were given
structureLabel() takes 3 positional arguments but 4 were given
structureLabel() takes 3 positional arguments but 4 were given
structureLabel() takes 3 positional arguments but 4 were given
structureLabel() takes 3 positional arguments but 4 were given
structureLabel() takes 3 positional arguments but 4 were given
structureLabel() takes 3 positional arguments but 4 were given
structureLabel() takes 3 positional arguments but 4 were given
structureLabel() takes 3 positional arguments but 4 were given
structureLabel() takes 3 positional arguments but 4 were given
structureLabel() takes 3 positional arguments but 4 were given
structureLabel() takes 3 positional arguments but 4 were given
structureLabel() takes 3 positional arguments but 4 were given
structureLabel() takes 3 positional arguments but 4 were given
structureLabel() takes 3 positional arguments but 4 wer

ITEM: R_02_578| ID: 642:  95%|█████████▌| 609/641 [00:16<00:02, 15.26it/s]

structureLabel() takes 3 positional arguments but 4 were given
structureLabel() takes 3 positional arguments but 4 were given


ITEM: R_01_577| ID: 645: 100%|██████████| 641/641 [00:17<00:00, 37.36it/s]

structureLabel() takes 3 positional arguments but 4 were given
structureLabel() takes 3 positional arguments but 4 were given
structureLabel() takes 3 positional arguments but 4 were given
structureLabel() takes 3 positional arguments but 4 were given
structureLabel() takes 3 positional arguments but 4 were given
structureLabel() takes 3 positional arguments but 4 were given
structureLabel() takes 3 positional arguments but 4 were given
structureLabel() takes 3 positional arguments but 4 were given
structureLabel() takes 3 positional arguments but 4 were given
structureLabel() takes 3 positional arguments but 4 were given
structureLabel() takes 3 positional arguments but 4 were given
structureLabel() takes 3 positional arguments but 4 were given
structureLabel() takes 3 positional arguments but 4 were given
structureLabel() takes 3 positional arguments but 4 were given
structureLabel() takes 3 positional arguments but 4 were given
structureLabel() takes 3 positional arguments but 4 wer

### Upload Image

In [37]:
item = 674 # The number according to the omeka s resource

In [ ]:
image = 'test.png'
imageType = 'png'

In [ ]:
data = {
    "o:ingester": "upload", 
    "file_index": "0", 
    "o:item": {"o:id": item }
}

files = [
     ('data', (None, json.dumps(data), 'application/json')),
     ('file[0]', (image, open(image, 'rb'), 'image/' + imageType))
]

response = requests.post(endpoint + '/media', params=params, files=files)